In [1]:
import pandas as pd
import numpy as np

In [17]:
dfSave = pd.read_csv('save2data.csv')
dfSave.head(10)

,gID,scoreDiff,inning,Current Outs,onFirst,onSecond,onThird,pID,name
0,ANA201004050,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
1,ANA201004050,3,9.0,1.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
2,ANA201004050,3,9.0,2.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
3,ANA201004230,2,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
4,ANA201004230,2,9.0,1.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
5,ANA201004230,2,9.0,2.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
6,ANA201004260,3,9.0,0.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
7,ANA201004260,3,9.0,1.0,NaN,NaN,NaN,fuenb001,Brian Fuentes
8,ANA201004260,3,9.0,1.0,hafnt001,NaN,NaN,fuenb001,Brian Fuentes
9,ANA201004260,3,9.0,2.0,hafnt001,NaN,NaN,fuenb001,Brian Fuentes


In [30]:
dfSave = dfSave.drop_duplicates(subset=['gID']).reset_index(drop=True)
dfSave.loc[(dfSave['inning'] <= 6) & (dfSave['scoreDiff'] >= 6) ]
print(len(dfSave))

3342
